In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/isic-2024-challenge/sample_submission.csv
/kaggle/input/isic-2024-challenge/train-metadata.csv
/kaggle/input/isic-2024-challenge/test-metadata.csv
/kaggle/input/isic-2024-challenge/test-image.hdf5
/kaggle/input/isic-2024-challenge/train-image.hdf5


KeyboardInterrupt: 

In [1]:
import numpy as np
import pandas as pd
import torch
import os
import torchmetrics
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from tqdm.notebook import tqdm_notebook as tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import h5py,io
import albumentations as A
from albumentations.pytorch import ToTensorV2

class HDF5Dataset(Dataset): #defining the dataset 
    def __init__(self, data, metadata, transform=None):
        self.data = h5py.File(data, 'r') #creating data argument
        if type(metadata) is pd.DataFrame:
            self.metadata = metadata
        else:
            self.metadata = pd.read_csv(metadata)
        self.transform = transform #creating transform argument 

    def __len__(self):
        return len(self.metadata) #returns dataset 

    def __getitem__(self, idx):#gets image and label 
        img_name = self.metadata.isic_id[idx] #accesses image filepath 
        image = np.array(self.data[img_name]) #opens image
        image = np.array(Image.open(io.BytesIO(image)),dtype=np.float32)/255

        label = int(self.metadata['target'].iloc[idx]) #find the label 

        if self.transform:
            augmented = self.transform(image=image) #transformation 
            image = augmented['image'] #grab the augmented image 

        return image, label #if you want to include metadata we would need to return more fields here   

dataset = '/kaggle/input/isic-2024-challenge/train-image.hdf5'
train_data = '/kaggle/input/isic-2024-challenge/train-metadata.csv'
# Train Transform can be tailored according to the needs. Or it can None
train_transform = A.Compose([
    A.Resize(height=224, width=224), #resize 
    A.OneOf([A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15),
             A.RandomBrightnessContrast() 
             ], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

#calls the custom dataset to access training data 
train_dataset = HDF5Dataset(dataset, train_data, transform=train_transform) 
#load training data 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4) 

/tmp/ipykernel_34/2814403026.py:13: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.metadata = pd.read_csv(metadata)


In [4]:
def conv_block(in_channels, out_channels, ker = 3, strd = 1, pad = 1,  pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=ker, stride = strd, padding=pad), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

In [5]:
class Resnet_10t(nn.Module):
    def __init__(self):
        super().__init__()
        self.b1 = nn.Sequential(
            conv_block(3, 24, strd = 2),
            conv_block(24, 32),
            conv_block(32, 64),
            nn.MaxPool2d(3, 2, 1)
        )
        self.b2 = nn.Sequential(
            conv_block(64, 64),
            conv_block(64, 64)
        )
        self.b3 = nn.Sequential(
            conv_block(64, 128, strd = 2),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128)
        )
        self.d1 = nn.Sequential(
            nn.AvgPool2d(2, 2),
            nn.Conv2d(64, 128, 1, stride = 1),
            nn.BatchNorm2d(128)
        )
        self.b4 = nn.Sequential(
            conv_block(128, 256, strd = 2),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256)
        )
        self.d2 = nn.Sequential(
            nn.AvgPool2d(2, 2),
            nn.Conv2d(128, 256, 1, stride = 1),
            nn.BatchNorm2d(256)
        )
        self.b5 = nn.Sequential(
            conv_block(256, 512, strd = 2),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512)
        )
        self.d3 = nn.Sequential(
            nn.AvgPool2d(2, 2),
            nn.Conv2d(256, 512, 1, stride = 1),
            nn.BatchNorm2d(512)
        )
        self.rel = nn.ReLU()
        self.khatam = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 2)
        )
    def forward(self, x):
        x = self.b1(x)
        x = self.b2(x)
        x1 = self.b3(x)
        x2 = self.d1(x)
        x = self.rel(x1 + x2)
        x1 = self.b4(x)
        x2 = self.d2(x)
        x = self.rel(x1 + x2)
        x1 = self.b5(x)
        x2 = self.d3(x)
        x = self.rel(x1 + x2)
        x = self.khatam(x)
        return x

In [6]:
cnn = Resnet_10t()
cnn.to(device)

Resnet_10t(
  (b1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (b2): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [7]:
lossfx = nn.CrossEntropyLoss()
lossfx.to(device)
optimizer = torch.optim.NAdam(cnn.parameters(), lr = 0.0005)

In [8]:
Epoch = 20
for epoch in tqdm(range(Epoch)):
    net_loss = 0.0
    cnn.train()
    for image, label in tqdm(train_loader):
#         x = data['image'].to(device)
#         y = data['label'].to(device)
        x = image.to(device)
        y = label.to(device)
        pred = cnn(x)
        loss_ = lossfx(pred, y)
        net_loss += loss_.item()
        optimizer.zero_grad()
        loss_.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} | Loss - {net_loss/len(train_loader)}")
#     mdl.eval()
#     with torch.inference_mode():
#         for data in tqdm(val_load):
#             x = data['image'].to(device)
#             y = data['label'].to(device)
#             pred = mdl(x)
#             acc.update(pred, y)
#         print(f"Validation Accuracy - {acc.compute()}")

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/12534 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
# from torch.utils.data import Dataset, DataLoader
# from PIL import Image
# import h5py,io
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# class HDF5Dataset(Dataset): #defining the dataset 
#     def __init__(self, data, metadata, transform=None):
#         self.data = h5py.File(data, 'r') #creating data argument
#         if type(metadata) is pd.DataFrame:
#             self.metadata = metadata
#         else:
#             self.metadata = pd.read_csv(metadata)
#         self.transform = transform #creating transform argument 

#     def __len__(self):
#         return len(self.metadata) #returns dataset 

#     def __getitem__(self, idx):#gets image and label 
#         img_name = self.metadata.isic_id[idx] #accesses image filepath 
#         image = np.array(self.data[img_name]) #opens image
#         image = np.array(Image.open(io.BytesIO(image)),dtype=np.float32)/255

# #         label = int(self.metadata['target'].iloc[idx]) #find the label 

#         if self.transform:
#             augmented = self.transform(image=image) #transformation 
#             image = augmented['image'] #grab the augmented image 

#         return image #if you want to include metadata we would need to return more fields here   
# test_dataset = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
# test_data = '/kaggle/input/isic-2024-challenge/test-metadata.csv'
# # Train Transform can be tailored according to the needs. Or it can None
# test_transform = A.Compose([
#     A.Resize(height=224, width=224), #resize 
#     # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
#     A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
#     ToTensorV2(),
# ])

# #calls the custom dataset to access training data 
# test_dataset = HDF5Dataset(test_dataset, test_data, transform=test_transform) 
# #load training data 
# test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False, num_workers=4)

In [17]:
# predictions = []

In [18]:
# cnn.eval()
# with torch.inference_mode():
#     for image in tqdm(test_loader):
#         x = image.to(device)
#         y = label.to(device)
#         pred = cnn(x)
#         predictions.append(pred)

In [19]:
# predictions

In [20]:
torch.save(cnn.state_dict(), 'resnet_10t_params.pth')